Reading notes and partial solutions to [Data Structures and Algorithms in Python](https://blackwells.co.uk/bookshop/product/9781118290279?gC=f177369a3b&gclid=Cj0KCQjwhJrqBRDZARIsALhp1WTBIyoxeQGXedlVy80vsglvFbNkVf7jTP0Z0zXEIP87lfqbtb4_diYaAr8dEALw_wcB).

In [1]:
import random
from matplotlib import pyplot as plt
%matplotlib inline
import math
from datetime import datetime
import time
import numpy as np

## Linked Lists

### Singly Linked Lists

In [15]:
class LinkedStack:
    
    class _Node:
        # to avoid auxiliary namespace dictionary
        # because there may be many nodes in a linked list
        __slots__ = '_element', '_next'

        def __init__(self, element, next):
            self._element = element
            self._next = next
    
    def __init__(self):
        self._head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def push(self, e):
        self._head = self._Node(e, self._head)
        self._size += 1
    
    def top(self):
        if self.is_empty():
            raise Exception('Stack is empty')
        return self._head._element
    
    def pop(self):
        if self.is_empty():
            raise Exception('Stack is empty')
        result = self._head._element
        self._head = self._head._next
        self._size -= 1
        return result

In [3]:
S = LinkedStack()
for i in [1,2,3]:
    S.push(i)
S.pop()

3

In [57]:
class LinkedQueue:
    
    class _Node:
        # to avoid auxiliary namespace dictionary
        # because there may be many nodes in a linked list
        __slots__ = '_element', '_next'

        def __init__(self, element, next):
            self._element = element
            self._next = next
    
    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element
    
    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        result = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty(): # if queue is now empty
            self._tail = None
        return result
    
    def enqueue(self, e):
        new = self._Node(e, self._head)
        if self.is_empty():
            self._head = new
        else:
            self._tail._next = new
        self._tail = new
        self._size += 1

### Circularly Linked Lists

In [22]:
class CircularQueue:
    
    class _Node:
        # to avoid auxiliary namespace dictionary
        # because there may be many nodes in a linked list
        __slots__ = '_element', '_next'

        def __init__(self, element, next):
            self._element = element
            self._next = next
    
    def __init__(self):
        self._tail = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Exception('Queue is empty')
        head = self._tail._next
        return head._element
    
    def dequeue(self):
        if self.is_empty():
            raise Exception('Queue is empty')
        head = self._tail._next
        if self._size == 1:
            self._tail = None
        else:
            self._tail._next = head._next
        self._size -= 1
        return head._element
    
    def enqueue(self, e):
        new = self._Node(e, None)
        if self.is_empty():
            new._next = new
        else:
            new._next = self._tail._next
            self._tail._next = new
        self._tail = new
        self._size += 1
    
    def rotate(self):
        if self._size > 0:
            self._tail = self._tail._next

In [24]:
q = CircularQueue()
for i in [1,2,3]:
    q.enqueue(i)
q.rotate()
print(q.first())

2


### Doubly Linked Lists

In [68]:
class _DoublyLinkedBase:
    
    class _Node:
        
        __slots__ = '_element', '_prev', '_next'
        
        def __init__(self, element, prev, next):
            self._element = element
            self._prev = prev
            self._next = next
    
        def __init__(self):
            self._header = self._Node(None, None, None)
            self._trailer = self._Node(None, None, None)
            self._header._next = self._trailer
            self._trailer._prev = self._header
            self._size = 0
        
        def __len__(self):
            return self._size
        
        def is_empty(self):
            return self._size == 0
        
        def _insert_between(self, e, predecessor, successor):
            new = self._Node(e, predecessor, successor)
            predecessor._next = new
            successor._prev = new
            self._size += 1
            return new
        
        def _delete_node(self, node):
            node._prev._next = node._next
            node._next._prev = node._prev
            self._size -= 1
            element = node._element
            node._prev = node._next = node_element = None
            return element

#### Deque

In [26]:
class LinkedDeque(_DoublyLinkedBase):
    def first(self):
        if self.is_empty():
            raise Exception('Queue is empty')
        return self._header._next._element
    
    def last(self):
        if self.is_empty():
            raise Exception('Queue is empty')
        return self._trailer._prev._element
    
    def insert_first(self, e):
        self._insert_between(e, self._header, self._header._next)
        
    def insert_last(self, e):
        self._insert_between(e, self._trailer._prev, self._trailer)
        
    def delete_first(self):
        if self.is_empty():
            raise Exception('Queue is empty')
        self._delete_node(self._header._next)
    
    def delete_last(self):
        if self.is_empty():
            raise Exception('Queue is empty')
        self._delete_node(self._trailer._prev)

#### Positional List

In [3]:
class PositionalList(_DoublyLinkedBase):
    
    class Position:
        
        def __init__(self, container, node):
            self._container = container
            self._node = node
        
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            """
            Return True if the two Positions represent the same location.
            """
            return type(other) is type(self) and other._node is self._node
        
        def __ne__(self, other):
            return not (self == other)
    
    def _validate(self, p):
        """
        Return the position's node, or raise error if the position is invalid.
        """
        if not isinstance(p, self.Position):
            raise TypeError('p must be of type Position')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._next is None:
            raise ValueError('p is deprecated')
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node."""
        if node is self._header or node is self._trailer:
            return None
        else:
            return self.Position(self, node)
    
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        """
        Return the Position before p.
        """
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        """
        Return the Position after p.
        """
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
    
    def _insert_between(self, e, predecessor, successor):
        """
        Add element between nodes and return new Position (instead of node as in the base class).
        """
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        node = self._validate(p)
        return self._insert_between(e, node._prev, node)
    
    def add_after(self, p, e):
        node = self._validate(p)
        return self._insert_between(e, node, node._next)
    
    def delete(self, p):
        node = self._validate(p)
        return self._delete_node(node)
        
    def replace(self, p, e):
        """
        Replace the element of the node at Position p with e.
        """
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

In [4]:
def insertion_sort(L):
    if len(L) > 1:
        marker = L.first()
        while marker != L.last():
            pivot = L.after(marker)
            value = pivot.element()
            if value > marker.element(): # pivot is already sorted
                marker = pivot
            else:
                # find leftmost item greater than value
                walk = marker
                while walk != L.first() and L.before(walk).element() > value:
                    walk = L.before(walk)
                L.delete(pivot)
                L.add_before(walk, value)

### Exercises

#### Reinforcement

**R-7.1** Give an algorithm for finding the second-to-last node in a singly linked
list in which the last node is indicated by a `next` reference of `None`.

In [10]:
class LinkedList:
    
    class Node:
        __slots__ = 'element', 'next'

        def __init__(self, element, next):
            self.element = element
            self.next = next
    
    def __init__(self):
        self.head = None
        self.tail = None
        self.size = 0
    
    def __len__(self):
        return self.size
    
    def add_first(self, e):
        new = self.Node(e, self.head)
        self.head = new
        self.size += 1
    
    def add_last(self, e):
        new = self.Node(e, None)
        if self.size == 0:
            self.head = new
        else:
            self.tail.next = new
        self.tail = new
        self.size += 1

In [11]:
def find_second_last(l):
    curr = l.head
    while curr.next.next is not None:
        curr = curr.next
    return curr.element

In [12]:
l = LinkedList()
for i in [1,2,3]:
    l.add_last(i)
find_second_last(l)

2

**R-7.2** Describe a good algorithm for concatenating two singly linked lists `L` and
`M`, given only references to the first node of each list, into a single list `L`
that contains all the nodes of `L` followed by all the nodes of `M`.

1. Traverse `L` starting from the head until reaching to the last node.
2. Assign the head of `M` to be the next node of the last node of `L`.

In [62]:
def concat(lHead, mHead):
    lLast = lHead
    while lLast.next is not None:
        lLast = lLast.next
    lLast.next = mHead

In [66]:
l = LinkedList()
m = LinkedList()
for i in [3,2,1]:
    l.addFirst(i)
for i in [5,4]:
    m.addFirst(i)
concat(l.head, m.head)

In [71]:
curr = l.head
while curr is not None:
    print(curr.element)
    curr = curr.next

1
2
3
4
5


**R-7.3** Describe a recursive algorithm that counts the number of nodes in a singly
linked list.

In [79]:
def count_helper(node, acc):
    if node is None:
        return acc
    else:
        return count_helper(node.next, acc + 1)

def count(l):
    return count_helper(l.head, 0)

In [80]:
l = LinkedList()
for i in [1,2,3]:
    l.addFirst(i)
count(l)

3

**R-7.4** Describe in detail how to swap two nodes `x` and `y` (and not just their contents) in a singly linked list `L` given references only to `x` and `y`. Repeat
this exercise for the case when `L` is a doubly linked list. Which algorithm
takes more time?

If `L` is singly linked:
1. Swap `x.next` with `y.next`.
2. Traverse `L` from its head to gain access to the predecessors of `x` and `y`, say `xPred`, `yPred`. Swap the reference to `x` from its predecessor, i.e., `xPrev.next`, with the reference to `y` with its predecessor, i.e., `yPrev.next`.
3. Swap `x.element` with `y.element`.

If `L` is doubly linked:
1. Swap `x.next` with `y.next`.
2. Swap `x.prev` with `y.prev`.
3. Swap `x.element` with `y.element`.

The first case takes more time because a node in a singly linked list has no access to its predecessor.

**R-7.5** Implement a function that counts the number of nodes in a circularly
linked list.

In [22]:
class CircularList:
    
    class Node:
        # to avoid auxiliary namespace dictionary
        # because there may be many nodes in a linked list
        __slots__ = 'element', 'next'

        def __init__(self, element, next):
            self.element = element
            self.next = next
    
    def __init__(self):
        self.head = None
        self.tail = None
        self.size = 0
    
    def __len__(self):
        return self.size
    
    def is_empty(self):
        return self.size == 0
    
    def add_first(self, e):
        new = self.Node(e, self.head)
        self.head = new
        if self.size == 0:
            self.tail = new
        else:
            self.tail.next = new
        self.size += 1
    
    def add_last(self, e):
        new = self.Node(e, self.head)
        if self.size > 0:
            self.tail.next = new
        else:
            self.head = new
        self.tail = new
        self.size += 1

In [25]:
c = CircularList()
for i in [1,2,3]:
    c.add_last(i)
curr = c.head
for i in range(9):
    print(curr.element)
    curr = curr.next

1
2
3
1
2
3
1
2
3


In [29]:
def count(c):
    # pretend we don't have the .size field
    head = c.head
    curr = head.next
    counter = 1
    while curr != head:
        counter += 1
        curr = curr.next
    return counter

In [32]:
c = CircularList()
for i in range(10):
    c.add_last(i)
count(c)

10

**R-7.6** Suppose that `x` and `y` are references to nodes of circularly linked lists,
although not necessarily the same list. Describe a fast algorithm for telling
if `x` and `y` belong to the same list.

Traverse nodes from `x` to see if can reach `y`.

In [48]:
def in_same_list(x, y):
    if x == y:
        return True
    curr = x.next
    while curr != x:
        if curr == y:
            return True
        curr = curr.next
    return False

In [49]:
A = CircularList()
for i in [1,2,3]:
    A.add_last(i)
B = CircularList()
for i in [4,5]:
    B.add_last(i)
x = A.head.next.next
y = B.head.next

In [50]:
in_same_list(x, y)

False

In [53]:
z = A.head.next
in_same_list(x, z)

True

In [52]:
z = x
in_same_list(x, z)

True

**R-7.7** Our CircularQueue class of Section 7.2.2 provides a `rotate()` method that
has semantics equivalent to `Q.enqueue(Q.dequeue())`, for a nonempty
queue. Implement such a method for the `LinkedQueue` class of Section 7.1.2 without the creation of any new nodes.

In [63]:
class LinkedQueue1(LinkedQueue):
    
    def rotate(self):
        self._tail._next = self._head
        self._tail = self._head
        self._head = self._head._next

In [67]:
q = LinkedQueue1()
for i in [1,2,3]:
    q.enqueue(i)
q.rotate()
while not q.is_empty():
    print(q.dequeue())

2
3
1


**R-7.8** Describe a nonrecursive method for finding, by link hopping, the middle
node of a doubly linked list with header and trailer sentinels. In the case
of an even number of nodes, report the node slightly left of center as the
"middle." (Note: This method must only use link hopping; it cannot use a
counter.) What is the running time of this method?

Traverse the doubly linked list from both ends (from the header and from the trailer) until the two cursors meet or are adjacent to each other.

Run time is $O(n)$.

In [72]:
class DoublyLinkedList:
    class Node:
        
        __slots__ = 'element', 'prev', 'next'
        
        def __init__(self, element, prev, next):
            self.element = element
            self.prev = prev
            self.next = next
    
    def __init__(self):
        self.header = self.Node(None, None, None)
        self.trailer = self.Node(None, None, None)
        self.header.next = self.trailer
        self.trailer.prev = self.header
        self.size = 0

    def __len__(self):
        return self.ize

    def is_empty(self):
        return self.size == 0

    def insert_between(self, e, predecessor, successor):
        new = self.Node(e, predecessor, successor)
        predecessor.next = new
        successor.prev = new
        self.size += 1
        return new

    def delete_node(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev
        self.size -= 1
        element = node.element
        node.prev = node.next = node.element = None
        return element
    
    def add_last(self, e):
        self.insert_between(e, self.trailer.prev, self.trailer)

In [77]:
def find_middle(dll):
    curr1 = dll.header
    curr2 = dll.trailer
    while curr1 != curr2 and curr1.next != curr2:
        curr1 = curr1.next
        curr2 = curr2.prev
    return curr1.element

In [83]:
dll = DoublyLinkedList()
for i in [1,2,3]:
    dll.add_last(i)
find_middle(dll)

2

In [87]:
dll = DoublyLinkedList()
for i in [1,2,3,4]:
    dll.add_last(i)
find_middle(dll)

2